In [3]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

import ujson as json
import pickle
import re
import malaya
from tqdm import tqdm

In [4]:
class Pointer:
    def __init__(self, filename):
        self.filename = filename
        self.index = 0

    def _save(self):
        with open(self.filename, 'wb') as fopen:
            pickle.dump(self.index, fopen)

    def increment(self):
        self.index += 1
        self._save()

    def load(self):
        if not os.path.exists(self.filename):
            return
        with open(self.filename, 'rb') as fopen:
            self.index = pickle.load(fopen)

In [5]:
filename = 'kelm_generated_corpus.jsonl'
pointer = Pointer(f'{filename}.pickle')
pointer.load()
index = 0
limit = 400000

In [6]:
results = []
with open(filename) as fopen:
    for line in fopen:
        if index >= pointer.index:
            l = json.loads(line)
            results.append(l)

            pointer.increment()
        index += 1
        if index == limit:
            break

In [7]:
len(results)

400000

In [8]:
partition_size = 5
limit = len(results) // partition_size
for i in range(partition_size):
    b = results[i * limit: limit + (i * limit)]
    with open(f'{filename}-{i}', 'w') as fopen:
        json.dump(b, fopen)